In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
# Importo base construida a partir de las Notas a los Estados Contables
# [Reserva por Pasivo Global], [Cantidad de Juicios Abiertos con Pagos Parciales] y [Total de Pagos de los Casos Abiertos con Juicios]

Dataset_NEECC=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Dataset_NEECC_Sinensup.xlsx', sheet_name = 'NEECC', usecols='A:G')

Dataset_NEECC=Dataset_NEECC.drop('des_cia',1)

In [4]:
# Importo y preparo Siniestros en Juicio y Pendiente de Liquidación de BALANCE_2

balances2_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\BALANCES_2.xlsx', usecols='A:F')
dataset_missing=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Missing_AM.xlsx', sheet_name='BALANCES_2')
balances2_df=balances2_df.append(dataset_missing)

#Filtro subramo y codigo de cuenta
balances2_df=balances2_df[(balances2_df.cod_subramo=='1.050.01')|(balances2_df.cod_subramo=='1.050.99')]
balances2A_df=balances2_df[balances2_df.cod_cuenta=='2.01.01.01.01.18.01.00']
balances2B_df=balances2_df[balances2_df.cod_cuenta=='2.01.01.01.01.18.02.00']

# Siniestros ART - En mediación
balances2A_df['Siniestros ART - En mediación']=balances2A_df['importe']
balances2A_df=balances2A_df.drop(['cod_cuenta', 'importe'],1)

# Siniestros ART - En juicio
balances2B_df['Siniestros ART - En juicio']=balances2B_df['importe']
balances2B_df=balances2B_df.drop(['cod_cuenta', 'importe'],1)

# Junto ambas variables y agrupo
BALANCES_2_df=balances2A_df.merge(balances2B_df, how='outer', on=['cod_cia','des_cia','año','cod_subramo'])
BALANCES_2_df_agrup=BALANCES_2_df.groupby(['cod_cia','año'], as_index=False)['Siniestros ART - En mediación','Siniestros ART - En juicio'].sum()

BALANCES_2_df_agrup['Periodo_balance']=BALANCES_2_df_agrup['año']
BALANCES_2_df_agrup=BALANCES_2_df_agrup.drop('año',1)
#BALANCES_2_df_agrup=BALANCES_2_df_agrup.drop('des_cia',1)

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
# Junto datasets: BALANCES2 + Sinensup

Dataset_Res41155=Dataset_NEECC.merge(BALANCES_2_df_agrup, how='left', on=['cod_cia','Periodo_balance'])

In [6]:
# Importo base construida a partir del SINENSUP
# [Saldo a amortizar dif. reserva pto. 33.4.1.6 (-)] y [Reserva Especial ART Res. 41.155]

Dataset_Sinensup=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Dataset_NEECC_Sinensup.xlsx', sheet_name = 'Sinensup')

Dataset_Sinensup=Dataset_Sinensup.drop('des_cia',1)

In [7]:
# Junto datasets: 

Dataset_Res41155=Dataset_Res41155.merge(Dataset_Sinensup, how='left', on=['cod_cia','Periodo_balance'])

In [8]:
# Importo y preparo Anexo 11

anexo11_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\ANEXO_11.xlsx', usecols='A,C,D,J')

dataset_missing=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Missing_AM.xlsx', sheet_name='ANEXO_11', usecols='A,C,D,J')

anexo11_df=anexo11_df.append(dataset_missing)

anexo11_df=anexo11_df[(anexo11_df.cod_subramo=='1.050.01')|(anexo11_df.cod_subramo=='1.050.99')]

anexo11_df_agrup=anexo11_df.groupby(['cod_cia','año'], as_index=False)['Conting_y_Desvios_de_Siniestralidad_ART'].sum()

anexo11_df_agrup['Periodo_balance']=anexo11_df_agrup['año']
anexo11_df_agrup=anexo11_df_agrup.drop('año',1)


In [9]:
# Junto datasets: Anexo 12 + Anexo 8 + Anexo 11

Dataset_Res41155=Dataset_Res41155.merge(anexo11_df_agrup, how='left', on=['cod_cia','Periodo_balance'])

In [10]:
Dataset_Res41155=Dataset_Res41155.replace(to_replace=np.nan, value=0)

In [11]:
# Importo nombre compañías

Dataset_compañías=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Compañías.xlsx', sheet_name='AM')

In [12]:
# Junto datasets: BALANCES_2 + Sinensup + NEEC + Compañías

Dataset_Res41155=Dataset_Res41155.merge(Dataset_compañías, how='left', on='cod_cia')

In [13]:
# Listo las etiquetas de las columnas

Dataset_Res41155=Dataset_Res41155.drop('des_cia',1)

cols = list(Dataset_Res41155.columns.values)
cols



['cod_cia',
 'Periodo_balance',
 'Cantidad de Juicios',
 'Reserva por Pasivo Global',
 'Cantidad de Juicios Abiertos con Pagos Parciales',
 'Total de Pagos de los Casos Abiertos con Juicios',
 'Siniestros ART - En mediación',
 'Siniestros ART - En juicio',
 'Saldo a amortizar dif. reserva pto. 33.4.1.6 (-)',
 'Reserva Especial ART Res. 41.155',
 'Conting_y_Desvios_de_Siniestralidad_ART',
 'denominacion_corta']

In [14]:
# Reordeno las columnas

reord=['cod_cia','denominacion_corta', 'Periodo_balance',  'Siniestros ART - En mediación',
 'Siniestros ART - En juicio',  'Saldo a amortizar dif. reserva pto. 33.4.1.6 (-)',
 'Reserva Especial ART Res. 41.155',
 'Cantidad de Juicios',
 'Reserva por Pasivo Global',
 'Cantidad de Juicios Abiertos con Pagos Parciales',
 'Total de Pagos de los Casos Abiertos con Juicios', 'Conting_y_Desvios_de_Siniestralidad_ART']
    
    
Dataset_Res41155=Dataset_Res41155[reord]

In [15]:
Dataset_Res41155.to_excel(r'C:\Users\eangenello\Desktop\Dt_Res41155.xlsx', index=False)

In [15]:
# Importo los datasets Sentencias de Primera Instancia (RENALI)

PRIM_INST=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\1-2019 SENTENCIAS.xlsx', sheet_name = 'Sent primera inst')

# Importo dataset de compañías para tener el cod_cia y la denominación corta

# OJO QUE AÚN NO APARECE MUTUAL PETROLEROS ART, CUANDO APAREZCA HAY QUE AGREGARLA AL DATASET EN "ASEGURADORA"

Dataset_compañías=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Compañías.xlsx', sheet_name='Sentencias')

PRIM_INST=PRIM_INST.merge(Dataset_compañías, how='inner', on='Aseguradora')

PRIM_INST=PRIM_INST.drop('Aseguradora',1)

PRIM_INST['Dataset']='Dataset_1ra_instancia'

In [16]:
# Creo variable Fecha
PRIM_INST['Fecha']=PRIM_INST['F_1ra_sent']

# Creo variable Monto
PRIM_INST['Monto']=PRIM_INST['MONTO PRIMERA INST']

# Creo variable instancia
PRIM_INST['Instancia']=PRIM_INST['Tipo sentencia 1° inst.']

# Filtro y me quedo son con los registros que necesito
PRIM_INST_filt=PRIM_INST[(PRIM_INST['Instancia']=='Hace lugar a la demanda totalmente.')|(PRIM_INST['Instancia']=='Hace lugar a la demanda parcialmente.')]

# Reemplazo
PRIM_INST_filt['Instancia']=['Demandas conciliadas' if PRIM_INST_filt['Tipo sentencia 1° inst.'][i]=='Homologación.' else 'Sentencia 1° instancia' for i in PRIM_INST_filt.index]

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
reord= [ 'cod_cia',
 'denominacion_corta',
 'N° EXP_JUDICIAL',
 'caratula',
 'F_notificacion',
 'CUIT EMPLEADOR',
 'OBJETO DE RECLAMO',
 'CONTINGENCIA RECLAMADA',
 'ENF. RECLAMADA',
 'MONTO RECLAMADO',
 'Tipo sentencia 1° inst.',
 'F_1ra_sent',
 'MONTO PRIMERA INST',
 '% PRIMERA INST',
 'Tipo sentencia 2° inst.',
 'f_2da_sent',
 'MONTO SEGUNDA INST',
 '% SEGUNDA INST',
 'Dataset',
 'Fecha',
 'Monto',
 'Instancia']

PRIM_INST_filt=PRIM_INST_filt[reord]

In [19]:
# Importo los datasets Sentencias de Primera Instancia (RENALI)

SEG_INST=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Sentencias_Renali\1-2019 SENTENCIAS.xlsx', sheet_name = 'Sent segunda inst')


In [20]:
SEG_INST=SEG_INST.merge(Dataset_compañías, how='inner', on='Aseguradora')

SEG_INST=SEG_INST.drop('Aseguradora',1)

SEG_INST['Dataset']='Dataset_2da_instancia'

In [21]:
# Creo variable Fecha
SEG_INST['Fecha']=SEG_INST['f_2da_sent']

# Creo variable Monto
SEG_INST['Monto']=SEG_INST['MONTO SEGUNDA INST']

# Creo variable instancia
SEG_INST['Instancia']=SEG_INST['Tipo sentencia 2° inst.']

# Filtro y me quedo son con los registros que necesito
SEG_INST_filt=SEG_INST[(SEG_INST['Instancia']=='Hace lugar a la demanda totalmente.')|(SEG_INST['Instancia']=='Hace lugar a la demanda parcialmente.')]

SEG_INST_filt['Instancia']='Sentencia 2° instancia'

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [22]:
# Listo las etiquetas de las columnas

cols = list(SEG_INST_filt.columns.values)
cols

['N° EXP_JUDICIAL',
 'caratula',
 'F_notificacion',
 'CUIT EMPLEADOR',
 'OBJETO DE RECLAMO',
 'CONTINGENCIA RECLAMADA',
 'ENF. RECLAMADA',
 'MONTO RECLAMADO',
 'Tipo sentencia 1° inst.',
 'F_1ra_sent',
 'MONTO PRIMERA INST',
 '% PRIMERA INST',
 'Tipo sentencia 2° inst.',
 'f_2da_sent',
 'MONTO SEGUNDA INST',
 '% SEGUNDA INST',
 'cod_cia',
 'denominacion_corta',
 'Dataset',
 'Fecha',
 'Monto',
 'Instancia']

In [23]:
reord= [ 'cod_cia',
 'denominacion_corta',
 'N° EXP_JUDICIAL',
 'caratula',
 'F_notificacion',
 'CUIT EMPLEADOR',
 'OBJETO DE RECLAMO',
 'CONTINGENCIA RECLAMADA',
 'ENF. RECLAMADA',
 'MONTO RECLAMADO',
 'Tipo sentencia 1° inst.',
 'F_1ra_sent',
 'MONTO PRIMERA INST',
 '% PRIMERA INST',
 'Tipo sentencia 2° inst.',
 'f_2da_sent',
 'MONTO SEGUNDA INST',
 '% SEGUNDA INST',
 'Dataset',
 'Fecha',
 'Monto',
 'Instancia']

SEG_INST_filt=SEG_INST_filt[reord]

In [24]:
# Junto Primera y Segunda Instancia

DT_Demandas_Instancias=PRIM_INST_filt.append(SEG_INST_filt)

In [25]:
DT_Demandas_Instancias.to_excel(r'C:\Users\eangenello\Desktop\Dt_Demandas_Instancias.xlsx', sheet_name='Dt_Demandas_Instancias', index=False)